# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []


        ####################################################################################################################
                                                # Edited Part 1/3 (start) #
        ####################################################################################################################
        self.alreday_guessed_letters = np.zeros(28, dtype=bool) # an array which signifies whether a letter is already guesses in a game

        self.unigram = np.zeros(28)   
        self.digram = np.zeros((28, 28))
        self.trigram = np.zeros((28, 28, 28))
        self.tetragram = np.zeros((28, 28, 28, 28))
        self.pentagram = np.zeros((28, 28, 28, 28, 28))
        self.build_grams()
        ####################################################################################################################
                                                # Edited Part 1/3 (finish) #
        ####################################################################################################################
   
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
    
####################################################################################################################################################
                                                # Edited Part 2/3 (start) #
####################################################################################################################################################


########################################################################
    # guess_letter function:
    def guess(self, word):
        word = '{' + word + '|'    # "{" in added in starting of word and "|" in the end
        word = word.replace(' ', '')  # remove all whitespaces  

        # initialize score array
        score = np.zeros(28)
        coeff = np.array([1,2,5,10,20])

        # calculate score of each char by different patterns of various length
        for i in range(5):
            score_ith = self.calculate_score(word = word, pattern_length = i+1) * coeff[i]
            # print(score_ith)
            score = score + score_ith
            # print(score)
        # score = self.normalized_probability(my_word,score)  # for unigram only
    
        guess_char = chr(score.argmax() + ord('a'))
        # print(score)
        self.alreday_guessed_letters[score.argmax()] = True
        return guess_char

###################################################################
# Calculate score array of each pattern #
###################################################################
    def calculate_score(self, word, pattern_length):
        score_ = np.zeros(28)

        index = 0
        while (index + pattern_length - 1) < len(word):
            word_subset = word[index : (index + pattern_length)]

            if word_subset.count('_') == 1:
                score_ = score_ + self.Conditional_Probability(word_subset)

            index = index + 1
        return score_


###############################################
# Build N grams (srart) #
###############################################
    def build_grams(self):
        for word in self.full_dictionary:
            word = '{' + word + '|'
            self.build_unigram(word)
            self.build_digram(word)
            self.build_trigram(word)
            self.build_tetragram(word)
            self.build_pentagram(word)
        # self.unigram = np.zeros(26)
    
    def build_unigram(self, word):
        unique_letters = np.zeros(28, dtype=bool)
        for char in word:
            a = int(ord(char) - ord("a"))
            unique_letters[a] = True
        for index in range(28):
            if unique_letters[index]:
                self.unigram[index] = self.unigram[index] + 1
        return self.unigram
    
    def build_digram(self, word):
        index = 0
        while index + 1 < len(word):
            char1 = ord(word[index]) - ord("a")
            char2 = ord(word[index + 1]) - ord("a")
            self.digram[char1, char2] = self.digram[char1, char2] + 1
            index = index + 1
    
    def build_trigram(self, word):
        index = 0
        while index + 2 < len(word):
            char1 = ord(word[index]) - ord("a")
            char2 = ord(word[index + 1]) - ord("a")
            char3 = ord(word[index + 2]) - ord("a")
            self.trigram[char1, char2, char3] = self.trigram[char1, char2, char3] + 1
            index = index + 1
    
    def build_tetragram(self, word):
        index = 0
        while index + 3 < len(word):
            char1 = ord(word[index]) - ord("a")
            char2 = ord(word[index + 1]) - ord("a")
            char3 = ord(word[index + 2]) - ord("a")
            char4 = ord(word[index + 3]) - ord("a")
            self.tetragram[char1, char2, char3, char4] = ( self.tetragram[char1, char2, char3, char4] + 1)
            index = index + 1
    
    def build_pentagram(self, word):
        index = 0
        while index + 4 < len(word):
            char1 = ord(word[index]) - ord("a")
            char2 = ord(word[index + 1]) - ord("a")
            char3 = ord(word[index + 2]) - ord("a")
            char4 = ord(word[index + 3]) - ord("a")
            char5 = ord(word[index + 4]) - ord("a")
            self.pentagram[char1, char2, char3, char4, char5] = (self.pentagram[char1, char2, char3, char4, char5] + 1)
            index = index + 1
###############################################
# Build N grams (End) #
###############################################


##########################################################
    # Calculating Probability (end) #
##########################################################
    
    #normalize the frequency of each letter which is not previously guessed
    def normalized_probability(self, letter_frequency):
        sum = 0
        # score = np.zeros(26)
        for i in range(28):
            if self.alreday_guessed_letters[i]:
                letter_frequency[i] = 0
            else:
                # score[i] = self.unigram[i]
                sum = sum + letter_frequency[i]
        
        if(sum<1): 
            #only true when finding individual letter probability in word(no grouping)
            sum = 0
            for j in range(28):
                if(self.alreday_guessed_letters[j]==False):
                    letter_frequency[j] = self.unigram[j]
                    sum = sum + self.unigram[j]
                    
            return letter_frequency/sum
        
        return letter_frequency/sum
    
    #probability of each letter which is not previously guessed
    def Conditional_Probability(self, word_subset):
        # subset has only 1 blank
        # finding probability of each letter in that blank
        length = len(word_subset)
        letter_frequency = np.zeros(28)

        # print (length)
        word_subset_original = word_subset
        for i in range(28):
            word_subset = word_subset_original
            if self.alreday_guessed_letters[i] == True:
                # letter_frequency[i] = 0
                continue

            # replace blank by ith letter and find its probaility
            word_subset = word_subset.replace('_', chr(ord("a") + i))
            # size 1
            # can directly can score from unigram table

            # size 2
            if (length == 2):
                char1 = ord(word_subset[0]) - ord("a")
                char2 = ord(word_subset[1]) - ord("a")
                letter_frequency[i] = self.digram[char1, char2]

            # size 3
            if length == 3:
                char1 = ord(word_subset[0]) - ord("a")
                char2 = ord(word_subset[1]) - ord("a")
                char3 = ord(word_subset[2]) - ord("a")
                letter_frequency[i] = self.trigram[char1, char2, char3]

            # size 4
            if length == 4:
                char1 = ord(word_subset[0]) - ord("a")
                char2 = ord(word_subset[1]) - ord("a")
                char3 = ord(word_subset[2]) - ord("a")
                char4 = ord(word_subset[3]) - ord("a")
                letter_frequency[i] = self.tetragram[char1, char2, char3,char4]

            # size 5
            if length == 5:
                char1 = ord(word_subset[0]) - ord("a")
                char2 = ord(word_subset[1]) - ord("a")
                char3 = ord(word_subset[2]) - ord("a")
                char4 = ord(word_subset[3]) - ord("a")
                char5 = ord(word_subset[4]) - ord("a")
                letter_frequency[i] = self.pentagram[char1, char2, char3, char4, char5]

        scor = self.normalized_probability(letter_frequency)
        # print(scor)
        return scor
##########################################################
    # Calculating Probability (end) #
##########################################################
    
####################################################################################################################################################
                                                # Edited Part 2/3 (end) #
####################################################################################################################################################


##########################################################
    # You'll likely not need to modify any of the code below #
##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []

####################################################################################################################################################
                                                # Edited Part 3/3 (start) #
####################################################################################################################################################

        self.alreday_guessed_letters = np.zeros(28, dtype=bool)
        self.alreday_guessed_letters[26] = True # "{" is not possible
        self.alreday_guessed_letters[27] = True  # "|" is not possible
####################################################################################################################################################
                                                # Edited Part 3/3 (end) #
####################################################################################################################################################


        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [4]:
api = HangmanAPI(access_token="34f6136e6c525eaec7ec254cd7c153", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [5]:
for i in range(20):
    api.start_game(practice=1,verbose=False)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


run 2057 practice games out of an allotted 100,000. practice success rate so far = 0.590
run 2058 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2059 practice games out of an allotted 100,000. practice success rate so far = 0.590
run 2060 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2061 practice games out of an allotted 100,000. practice success rate so far = 0.590
run 2062 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2063 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2064 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2065 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2066 practice games out of an allotted 100,000. practice success rate so far = 0.590
run 2067 practice games out of an allotted 100,000. practice success rate so far = 0.589
run 2068 practice gam

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [6]:
# for i in range(1000):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     #api.start_game(practice=0,verbose=False)
    
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [7]:
# [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# success_rate = total_recorded_successes/total_recorded_runs
# print('overall success rate = %.3f' % success_rate)